### Predicting  California House Prices with  Linear Regression

### Objective
* To predict California Housing Prices using the most simple Linear Regression Model and see how it performs.
* To understand the modeling workflow using mlpack.

### About the Data
 This dataset is a modified version of the California Housing dataset available from Luís Torgo's page (University of Porto). Luís Torgo obtained it from the StatLib repository (which is closed now). The dataset may also be downloaded from StatLib mirrors.
 
 This dataset is also used in a book HandsOn-ML (a very good book and highly recommended: https://www.oreilly.com/library/view/hands-on-machine-learning/9781491962282/]).
 
 The dataset in this directory is almost identical to the original, with two differences:
207 values were randomly removed from the totalbedrooms column, so we can discuss what to do with missing data. An additional categorical attribute called oceanproximity was added, indicating (very roughly) whether each block group is near the ocean, near the Bay area, inland or on an island. This allows discussing what to do with categorical data.
Note that the block groups are called "districts" in the Jupyter notebooks, simply because in some contexts the name "block group" was confusing."

Lets look at the features of the dataset:
* Longitude : Longitude coordinate of the houses.
* Latitude : Latitude coordinate of the houses.
* Housing Median Age : Average lifespan of houses.
* Total Rooms : Number of rooms in a location.
* Total Bedrooms : Number of bedroooms in a location.
* Population : Population in that location.
* Median Income : Median Income of households in a location.
* Median House Value : Median House Value in a location.
* Ocean Proximity : Closeness to shore. 

### Approach
 Here, we will try to recreate the workflow from the book mentioned above. 
 * Look at the Big Picture.
 * Get the Data.
 * Discover and Visualize the data to gain insights.
 * Pre-Process the data for the Ml Algorithm.
 * Create new features. 
 * Splitting the data.
 * Training the ML model using MLPACK.
 * Residuals, Errors and Conclusion.


### Big Picture

Suppose you work in a Real State Agency as an analyst or Data Scientist and your Boss wants you to predict the housing prices in a certain location. You are provided with a dataset. So, what will be the first thing to do?

If you are probably jumping right into analysing the data and ML algorithms, then this is not the right way to go.
<h5>The first thing is to Ask Questions.</h5>
 
Questions like:
  * What will be the predictions used for?
  * Will it be fed into some other system or not?
and many more, just to have concrete goals.
 
In this case, your boss says that they will be using the data to get the predictions so that the other team can work on some investment strategies.
 
So, let's get started.

<h3> Importing Header Files </h3>

In [1]:
#include <mlpack/xeus-cling.hpp>
#include <mlpack.hpp>

In [2]:
#define WITHOUT_NUMPY 1
#include "matplotlibcpp.h"
#include "xwidgets/ximage.hpp"

/* CPython Api Scripts for Plots */

#include "../utils/histogram.hpp"
#include "../utils/impute.hpp"
#include "../utils/pandasscatter.hpp"
#include "../utils/heatmap.hpp"
#include "../utils/plot.hpp"

namespace plt = matplotlibcpp;

In [3]:
using namespace mlpack;
using namespace mlpack::data;

<h3> Let's download the dataset. </h3>

In [4]:
!wget -q https://datasets.mlpack.org/examples/housing.csv

### Loading the Data
Now, we need to load the dataset as an Armadillo matrix for further operations. Our dataset has a total of 9 features: 8 numerical and 1 categorical(ocean proximity). We need to map the categorical features, as armadillo operates on numeric values only.

But, there's one thing which we need to do before loading the dataset as an Armadillo matrix; that is, we need to deal with any missing values. Since 207 values were removed from the original dataset from "total_bedrooms_column", we need to fill them using either "mean" or "median" of that feature (for numerical) and "mode" (for categorical).

In [5]:
// The imputing functions follows this:
// Impute(inputFile, outputFile, kind);
// Here, inputFile is our raw file, outputFile is our new file with the imputations, 
// and kind refers to imputation method.

Impute("housing.csv", "housing_imputed.csv", "median");

Let's drop the headers using sed. Sed is a unix utility which is used to parse and transform text.

In [6]:
!sed 1d housing_imputed.csv > housing_without_header.csv

// Here, we used sed to delete the first row which is indicated by "1d" and created a new file with name
// housing_without_header.csv

In [7]:
arma::mat dataset;
data::DatasetInfo info;
info.Type(9) = mlpack::data::Datatype::categorical;
data::Load("housing_without_header.csv", dataset, info);

In [8]:
// Print the first 6 rows of the input data.
std::cout << dataset.submat(0, 0, dataset.n_rows - 1 , 5)<< std::endl;

  -1.2223e+02  -1.2222e+02  -1.2224e+02  -1.2225e+02  -1.2225e+02  -1.2225e+02
   3.7880e+01   3.7860e+01   3.7850e+01   3.7850e+01   3.7850e+01   3.7850e+01
   4.1000e+01   2.1000e+01   5.2000e+01   5.2000e+01   5.2000e+01   5.2000e+01
   8.8000e+02   7.0990e+03   1.4670e+03   1.2740e+03   1.6270e+03   9.1900e+02
   1.2900e+02   1.1060e+03   1.9000e+02   2.3500e+02   2.8000e+02   2.1300e+02
   3.2200e+02   2.4010e+03   4.9600e+02   5.5800e+02   5.6500e+02   4.1300e+02
   1.2600e+02   1.1380e+03   1.7700e+02   2.1900e+02   2.5900e+02   1.9300e+02
   8.3252e+00   8.3014e+00   7.2574e+00   5.6431e+00   3.8462e+00   4.0368e+00
   4.5260e+05   3.5850e+05   3.5210e+05   3.4130e+05   3.4220e+05   2.6970e+05
            0            0            0            0            0            0



Did you notice something? Yes, the last row looks like it is entirely filled with '0'. Let's check our dataset to see what it corresponds to.
It corresponds to Ocean Proximity which is a categorical value, but here it is zero.
Why? It's because the load function loads numerical values only. This is exactly why we mapped Ocean proximity earlier.
So, let's deal with this.

In [9]:
arma::mat encoded_dataset; 
data::OneHotEncoding(dataset, encoded_dataset, info);

Here, we chose our pre-built encoding method "One Hot Encoding" to deal with the categorical values.

In [10]:
encoded_dataset.n_rows
// The above code prints the number of rows(features + labels) in current dataset.

14

You can notice the number of rows changed from 10 to 14, siginifying one hot encoding in this case.

<h3>Visualization</h3>

Let's plot a histogram. 

In [11]:
// Hist(inputFile, bins, width, height, outputFile);
Hist("housing.csv", 50, 20, 15, "histogram.png");
auto im = xw::image_from_file("histogram.png").finalize();
im

A Jupyter widget with unique id: a5f1f1effcb642b195677db04a6f597c

Let's plot a scatter plot with longitude and latitude as x and y coordinates respectively.

In [12]:
// PandasScatter(inputFile, x, y, outputFile);
PandasScatter("housing.csv", "longitude", "latitude", "output.png");
auto im = xw::image_from_file("output.png").finalize();
im

A Jupyter widget with unique id: ba73ba78ad5c4052b9a811fd9e08db7c

Let's add some colour to the scatter plot.

In [13]:
// PandasScatterColor(inputFile, x, y, label, c, outputFile);
PandasScatterColor("housing.csv","longitude","latitude","Population","median_house_value","output1.png");
auto im = xw::image_from_file("output1.png").finalize();
im

A Jupyter widget with unique id: 2192215b3cbc4752a0160f4594594e01

Let's take it a step further and plot this on top of California map.

In [14]:
//PandasScatterMap(inputFile, imgFile, x, y, label, c, outputFile);
PandasScatterMap("housing.csv","california.png","longitude","latitude","Population","median_house_value","output2.png");
auto im = xw::image_from_file("output2.png").finalize();
im

A Jupyter widget with unique id: 38cffa6bcbdd41fa9b20b1f8ef9b2a21

<h3>Correlation</h3>

In [15]:
// HeatMap(inputFile, outputFile);
HeatMap("housing.csv", "heatmap.png");
auto im = xw::image_from_file("heatmap.png").finalize();
im

A Jupyter widget with unique id: e9f1b75bb18e41a8876ff8fe24437a63

<h3>Train-Test Split</h3>
The dataset needs to be split into a training and testing set before we learn any model.

In [16]:
// Labels are median_house_value which is row 8
arma::rowvec labels =
    arma::conv_to<arma::rowvec>::from(encoded_dataset.row(8));
encoded_dataset.shed_row(8);

In [17]:
arma::mat trainSet, testSet;
arma::rowvec trainLabels, testLabels;

In [18]:
// Split dataset randomly into training set and test set.
data::Split(encoded_dataset, labels, trainSet, testSet, trainLabels, testLabels,
    0.2 /* Percentage of dataset to use for test set. */);

### Training the linear model

Regression analysis is the most widely used method of prediction. Linear regression is used when the dataset has a linear correlation and as the name suggests, multiple linear regression has one independent variable (predictor) and one or more dependent variable(response).

The simple linear regression equation is represented as y = $a + b_{1}x_{1} + b_{2}x_{2} + b_{3}x_{3} + ... + b_{n}x_{n}$ where $x_{i}$ is the ith explanatory variable, y is the dependent variable, $b_{i}$ is ith coefficient and a is the intercept.

To perform linear regression we'll be using the `LinearRegression` class from mlpack.

In [19]:
LinearRegression lr(trainSet, trainLabels, 0.5);
// The line above creates and train the model.

In [20]:
// Let's create a output vector for storing the results.
arma::rowvec output; 
lr.Predict(testSet, output);

In [21]:
lr.ComputeError(trainSet, trainLabels);

In [22]:
std::cout << lr.ComputeError(trainSet, trainLabels);

4.70395e+09

Let's manually check some predictions.

In [23]:
std::cout << testLabels[1] << std::endl;
std::cout << output[1] << std::endl;

158000
257831


In [24]:
std::cout << testLabels[7] << std::endl;
std::cout << output[7] << std::endl;

127400
218479


In [25]:
arma::mat preds;
preds.insert_rows(0, testLabels);
preds.insert_rows(1, output);

arma::mat diffs = preds.row(1) - preds.row(0);

In [26]:
data::Save("./preds.csv", preds);
data::Save("./predsDiff.csv", diffs);

In [27]:
!sed -i '1iY_Test,Y_Preds' ./preds.csv

In [28]:
!sed -i '1iY_Diff' ./predsDiff.csv

### Model Evaluation
Test data is visualized with `testLabels` and `output`, the blue points indicates the data points and the blue line indicates the regression line or best fit line.

In [29]:
LmPlot("./preds.csv", "Y_Test", "Y_Preds", "predsScatter");
auto img = xw::image_from_file("plots/predsScatter.png").finalize();    
img

A Jupyter widget with unique id: 3d016d5b244f4da0a8ffd92e7aafeeae

In [30]:
HistPlot("./predsDiff.csv", "Y_Diff", "Distribution of residuals");
auto img = xw::image_from_file("plots/Distribution of residuals.png").finalize();
img

A Jupyter widget with unique id: 9bf4813e8c894e6cb4de110544bf7b50

## Evaluation Metrics for Regression model

In the previous cell we have visualized our model performance by plotting the best fit line. Now we will use various evaluation metrics to understand how well our model has performed.

* Mean Absolute Error (MAE) is the sum of absolute differences between actual and predicted values, without considering the direction.
$$ MAE = \frac{\sum_{i=1}^n\lvert y_{i} - \hat{y_{i}}\rvert} {n} $$
* Mean Squared Error (MSE) is calculated as the mean or average of the squared differences between predicted and expected target values in a dataset, a lower value is better
$$ MSE = \frac {1}{n} \sum_{i=1}^n (y_{i} - \hat{y_{i}})^2 $$
* Root Mean Squared Error (RMSE), Square root of MSE yields root mean square error (RMSE) it indicates the spread of the residual errors. It is always positive, and a lower value indicates better performance.
$$ RMSE = \sqrt{\frac {1}{n} \sum_{i=1}^n (y_{i} - \hat{y_{i}})^2} $$

In [31]:
// Model evaluation metrics.

std::cout << "Mean Absolute Error: " << arma::mean(arma::abs(output - testLabels)) << std::endl;
std::cout << "Mean Squared Error: " << arma::mean(arma::pow(output - testLabels,2)) << std::endl;
std::cout << "Root Mean Squared Error: " << sqrt(arma::mean(arma::pow(output - testLabels,2))) << std::endl;

Mean Absolute Error: 50816.8
Mean Squared Error: 4.94139e+09
Root Mean Squared Error: 70295


We can clearly see that the MAE is 49674; when we compare this with the median house value, it doesn't seem to be a very good fit. 

Thus we can conclude that, the simple Linear Regression models is not being able to catch all the features.
So, maybe it's time to try other algorithms. 

<h5>NOTE:</h5> In the entire ML workflow, you never know exactly which model will perform the best. So, usually you try a lot of different algorithms to see which fits the model.